In [1]:
# Add this to the END of your /home/checker/jetracer/jetracer/camera_utils.py file
# Then reinstall the package

import gc
import time

def release_all_cameras():
    """
    Universal camera release function - call this instead of restarting kernel
    """
    print("🔄 Releasing all cameras...")
    
    # Get globals from the calling frame
    import inspect
    frame = inspect.currentframe().f_back
    caller_globals = frame.f_globals
    
    # Step 1: Find and stop all camera objects
    camera_vars = []
    for var_name in list(caller_globals.keys()):
        try:
            obj = caller_globals[var_name]
            # Check for camera-like objects
            if (hasattr(obj, 'running') or 
                hasattr(obj, 'cap') or 
                'camera' in str(type(obj)).lower() or
                'CSI' in str(type(obj))):
                camera_vars.append(var_name)
        except:
            pass
    
    print(f"Found camera objects: {camera_vars}")
    
    # Step 2: Stop all cameras
    for var_name in camera_vars:
        try:
            obj = caller_globals[var_name]
            if hasattr(obj, 'running'):
                obj.running = False
                print(f"✓ Stopped {var_name}.running")
            if hasattr(obj, 'stop'):
                obj.stop()
                print(f"✓ Called {var_name}.stop()")
            if hasattr(obj, 'cap') and hasattr(obj.cap, 'release'):
                obj.cap.release()
                print(f"✓ Released {var_name}.cap")
            # Delete the variable
            del caller_globals[var_name]
            print(f"✓ Deleted {var_name}")
        except Exception as e:
            print(f"Warning cleaning {var_name}: {e}")
    
    # Step 3: Force garbage collection
    gc.collect()
    print("✓ Garbage collection completed")
    
    # Step 4: Wait for hardware to fully release
    print("⏳ Waiting for camera hardware to release...")
    time.sleep(3)
    
    print("✅ Camera release complete!")

def quick_camera_test():
    """Quick test to verify camera is available"""
    try:
        from jetcam.csi_camera import CSICamera
        test_cam = CSICamera(width=640, height=480, capture_fps=21)
        test_cam.running = True
        time.sleep(2)
        
        img = test_cam.value
        if img is not None:
            print(f"✅ Camera available: {img.shape}")
            result = True
        else:
            print("❌ Camera not capturing")
            result = False
            
        test_cam.running = False
        del test_cam
        return result
        
    except Exception as e:
        print(f"❌ Camera test failed: {e}")
        return False

def safe_camera_create(camera_class, *args, **kwargs):
    """Safely create a camera - releases any existing cameras first"""
    print("🔄 Safe camera creation...")
    
    # Release any existing cameras
    release_all_cameras()
    
    # Create new camera
    try:
        camera = camera_class(*args, **kwargs)
        print(f"✅ Created {camera_class.__name__}")
        return camera
    except Exception as e:
        print(f"❌ Failed to create {camera_class.__name__}: {e}")
        return None

# Convenient shortcuts
def release_cam():
    """Short alias for release_all_cameras()"""
    release_all_cameras()

def test_cam():
    """Short alias for quick_camera_test()"""
    return quick_camera_test()